# 카카오맵 API
### 도로명주소로부터 위도, 경도 정보 구하기

In [3]:
import requests
from urllib.parse import quote
import pandas as pd

In [4]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [5]:
addr = '서울특별시 강서구 화곡로 302 (화곡동)'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)
result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
result

{'documents': [{'address': {'address_name': '서울 강서구 화곡동 980-16',
    'b_code': '1150010300',
    'h_code': '1150059100',
    'main_address_no': '980',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '강서구',
    'region_3depth_h_name': '화곡6동',
    'region_3depth_name': '화곡동',
    'sub_address_no': '16',
    'x': '126.849532173376',
    'y': '37.5509655144007'},
   'address_name': '서울 강서구 화곡로 302',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 강서구 화곡로 302',
    'building_name': '강서구청',
    'main_building_no': '302',
    'region_1depth_name': '서울',
    'region_2depth_name': '강서구',
    'region_3depth_name': '화곡동',
    'road_name': '화곡로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.849532173376',
    'y': '37.5509655144007',
    'zone_no': '07658'},
   'x': '126.849532173376',
   'y': '37.5509655144007'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [6]:
result['documents'][0]['x']

'126.849532173376'

In [7]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.5509655144007, 126.849532173376)

### address.csv 파일 읽어서 위도, 경도 정보 추가하기

In [8]:
df = pd.read_csv('address.csv')
df

,bldg,road_addr
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동)
1,양천구청,서울특별시 양천구 목동동로 105 (신정동)
2,부천시청,경기도 부천시 길주로 210-2 (중동)
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가)
4,김포시청,경기도 김포시 사우중로 1 (사우동)


In [9]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [10]:
lat_list = []; lng_list = []
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df['road_addr'][i])
    result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [11]:
df['lat'] = lat_list
df['lng'] = lng_list
df

,bldg,road_addr,lat,lng
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동),37.550966,126.849532
1,양천구청,서울특별시 양천구 목동동로 105 (신정동),37.517075,126.866543
2,부천시청,경기도 부천시 길주로 210-2 (중동),37.502277,126.765004
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가),37.525963,126.896367
4,김포시청,경기도 김포시 사우중로 1 (사우동),37.615190,126.715717


In [12]:
df.to_csv('address2.csv', sep=',', encoding='utf8', index=False)